In [1]:
import pandas as pd

students_df = pd.read_csv('resource/students.csv')
print(students_df.shape)

(24, 4)


In [2]:
students_df.head()

,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1


In [3]:
cars_df = pd.read_csv('resource/cars.csv')
print(cars_df.shape)

(6, 2)


In [4]:
cars_df

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


In [5]:
import pandas as pd
import pulp

In [8]:
students_df = pd.read_csv('resource/students.csv')
cars_df = pd.read_csv('resource/cars.csv')

prob = pulp.LpProblem('ClubCarProblem', pulp.LpMinimize)

S = students_df['student_id'].to_list()
C = cars_df['car_id'].to_list()
G = [1, 2, 3, 4]
SC = [(s,c) for s in S for c in C]
S_license = students_df[students_df['license']==1]['student_id']
S_g = {g: students_df[students_df['grade'] == g]['student_id'] for g in G}
S_male = students_df[students_df['gender']==0]['student_id']
S_female = students_df[students_df['gender']==1]['student_id']

U = cars_df['capacity'].to_list()

x = pulp.LpVariable.dicts('x', SC, cat='Binary')

for s in S:
  prob += pulp.lpSum([x[s,c] for c in C]) == 1

for c in C:
  prob += pulp.lpSum([x[s,c] for s in S]) <= U[c]

for c in C:
  prob += pulp.lpSum([x[s,c] for s in S_license]) >= 1

for c in C:
  for g in G:
    prob += pulp.lpSum([x[s,c] for s in S_g[g]]) >= 1

for c in C:
  prob += pulp.lpSum([x[s,c] for s in S_male]) >= 1

for c in C:
  prob += pulp.lpSum([x[s,c] for s in S_female]) >= 1

In [9]:
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/9ba88157b75f48b1a12f88050498c7f4-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/9ba88157b75f48b1a12f88050498c7f4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 77 COLUMNS
At line 1039 RHS
At line 1112 BOUNDS
At line 1258 ENDATA
Problem MODEL has 72 rows, 145 columns and 672 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 24 SOS with 144 members
Cgl0004I processed model has 72 rows, 144 columns (144 integer (144 of which binary)) and 672 elements
Cbc0038I Initial state - 29 integers unsatisfied sum - 8.8
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 0 ite

In [10]:
car2students = {c: [ s for s in S if x[s,c].value()==1] for c in C}

max_people = dict(zip(cars_df['car_id'], cars_df['capacity']))

for s, ss in car2students.items():
  print(f"車ID:{c}")
  print(f"学生数（乗車定員）:{len(ss)}({max_people[c]})")
  print(f"学生:{ss}\n")

車ID:5
学生数（乗車定員）:4(5)
学生:[6, 7, 8, 9]

車ID:5
学生数（乗車定員）:4(5)
学生:[4, 18, 21, 23]

車ID:5
学生数（乗車定員）:4(5)
学生:[5, 19, 20, 22]

車ID:5
学生数（乗車定員）:4(5)
学生:[3, 10, 13, 16]

車ID:5
学生数（乗車定員）:4(5)
学生:[0, 2, 11, 17]

車ID:5
学生数（乗車定員）:4(5)
学生:[1, 12, 14, 15]

